A way to inherit options from parts by specifying the options in a Meta class inside the part.

In [1]:
import logging

from donkeycar.log import get_logger
logger = get_logger(__name__)
logger.setLevel(logging.DEBUG)

using donkey v2.5.1 ...


In [2]:
logger.warning('test')


test


In [40]:
class PartMeta(type):
    """ metaclass for Parts """
    def __new__(cls, name, parents, dct):

        #set meta vars of parents first
        def get_parent_meta(parents, attrs=dict()):
            for p in parents:
                #print('parrent:', p)
                meta = getattr(p, 'Meta', None)
                if meta is not None:
                    attrs.update(meta.__dict__)
                    #print('attrs: ', attrs)
                    #print('type p', type(p))
                    if len(p.__bases__) > 0:
                        attrs = get_parent_meta(p.__bases__, attrs)
            return attrs
                    
        meta = get_parent_meta(parents)
                
        #override the Meta options with 
        this_meta = dct.get('Meta', dict())
        #print(this_meta)
        if this_meta is not None:
            meta.update(this_meta.__dict__)
            
        meta = dict([(k, v) for k,v in meta.items() if not k.startswith('_')])
        
        
        dct['Meta'] = type("Meta", (), meta)
        # we need to call type.__new__ to complete the initialization
        return super(PartMeta, cls).__new__(cls, name, parents, dct)
    
    def __init__(cls, name, parents, dct):
        cls.options = dict([(k, v) for k,v in cls.Meta.__dict__.items() if not k.startswith('_')])
        print('cls.options: ', cls.options)

In [41]:
class Part(object, metaclass=PartMeta):
    class Meta:
        pass
    
    def __init__(self, *args, **kwargs):
        print(self.options)
        print('Part loaded')
        self.options.update(**kwargs)
        print(kwargs)
        

        
print(Part.Meta)

cls.options:  {}
<class '__main__.Meta'>


In [42]:
class P1(Part):
    class Meta:
        X = 1
        p1 = True
print(P1.Meta)

cls.options:  {'X': 1, 'p1': True}
<class '__main__.Meta'>


In [51]:
class P2(P1):
    class Meta:
        X = 2
        p2 = True
        
    def __init__(self):
        p2 = self.options.get('p2')
        
print(P2.Meta.__dict__)


cls.options:  {'X': 2, 'p1': True, 'p2': True}
{'X': 2, 'p1': True, 'p2': True, '__module__': '__main__', '__dict__': <attribute '__dict__' of 'Meta' objects>, '__weakref__': <attribute '__weakref__' of 'Meta' objects>, '__doc__': None}


In [52]:
p = P2()

In [53]:
p.options

{'X': 2, 'p1': True, 'p2': True}

In [55]:
p.Meta.__dict__

mappingproxy({'X': 2,
              'p1': True,
              'p2': True,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Meta' objects>,
              '__weakref__': <attribute '__weakref__' of 'Meta' objects>,
              '__doc__': None})

In [7]:
p2 = P2(max=123123)
p2.options

{'X': 2, 'p1': True, 'p2': True}
Part loaded
{'max': 123123}


{'X': 2, 'p1': True, 'p2': True, 'max': 123123}

In [8]:
p2.options

{'X': 2, 'p1': True, 'p2': True, 'max': 123123}

In [9]:
config_example = \
{
    'parts': [
        {'path':'donkeycar.parts.clock.Timestamp',
        }
    ]
}

In [10]:
def import_class(cl):
    d = cl.rfind(".")
    classname = cl[d+1:len(cl)]
    mod_name = cl[0:d]
    print('{}, {}'.format(classname, mod_name))
    m = __import__(mod_name, globals(), locals(), [classname])
    return getattr(m, classname)

In [13]:
class PartManager:
    def __init__(self, ):
        self.parts = []
    
    def load_config(self, config_dict):
        parts_config = config_dict.get('parts', {})
        for d in parts_config:
            print(d)
            part = self.load_part(d.get('path'), options_dict=d.get('options', {}) )
            self.parts.append(part)
    
    def load_part(self, part_string, options_dict=dict()):
        #print(options_dict)
        part_class = import_class(part_string)
        part_instance = part_class(**options_dict)
        return part_instance
    
    def run_active_parts(self):
        pass
    
    def activate_part(self, part):
        pass
    

    

In [14]:
pl = PartManager()
#pt = pl.load_part('donkeycar.parts.clock.Timestamp')
pl.load_config(config_example)

{'path': 'donkeycar.parts.clock.Timestamp'}
Timestamp, donkeycar.parts.clock


In [15]:
pl.parts

In [17]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# This returns a tensor
inputs = Input(shape=(1,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

In [18]:
model.predict([3])

array([[0.05832965, 0.07807902, 0.10276888, 0.06221944, 0.1308823 ,
        0.1193485 , 0.12378903, 0.09922621, 0.09842636, 0.12693061]],
      dtype=float32)

In [63]:
import os
mpath = os.path.expanduser('~/test_save.h5')
model.save(mpath)
m2 = keras.models.load_model(mpath)

In [64]:
from tensorflow import keras

In [67]:
m2.predict([3])

array([[0.0936005 , 0.131739  , 0.05327277, 0.10235716, 0.08597475,
        0.07081458, 0.08201193, 0.07746188, 0.07892269, 0.22384475]],
      dtype=float32)